This notebook does the following:

Recommend tags based on Tag Application history, for user 43715

1) # Tag Applications
    * user-tag-#apps
    * order by #apps
    * weighted by total num apps


In [3]:
import pandas as pd
import numpy as np

In [4]:
# Connect to the database
import pymysql.cursors
import json

# Connect to the database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_mirror")

try:
    with connection.cursor() as cursor:
        # This query select rows from log_trailer_actions where the action is either TrailerModalLaunched or Recommendations Refreshed
        # These two actions indicate that a recommendation list is generated. 
        
        sql = """
        SELECT userId, movieId, tag, affect, tstamp
        FROM tag_events
        WHERE userId = 313322
        """ 
        
        cursor.execute(sql)
        result1 = cursor.fetchall()
            
finally: 
    connection.close()


In [5]:
user_tag_app = pd.DataFrame(list(result1), columns=['userId','movieId','tag','affect','tstamp'])

In [6]:
# Count the number of times user applied a tag on a certain movie. 
# If we take the sum, that will be the # times user applied this tag

user_tag_app2 = user_tag_app.groupby(['tag','movieId']).count().reset_index()
# user_tag_app2.drop(columns=['tstamp'],inplace=True)
user_tag_app2.rename(columns={'userId':"user_num_app"},inplace=True)
user_tag_app2.head()

,tag,movieId,user_num_app,affect,tstamp
0,Hayao Miyazaki,3000,1,1,1
1,Hayao Miyazaki,8253,1,1,1
2,dystopia,3000,1,1,1
3,nature,3000,1,1,1
4,nature vs. civilization,3000,1,1,1


In [7]:
# This block counts how many times a tag has been applied; 
# Used for weighting user tags

import pymysql.cursors
import json

# Connect to the database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_mirror")
try:
    with connection.cursor() as cursor:
            sql = "SELECT * FROM tag_movie"
            cursor.execute(sql)
            result = cursor.fetchall()
            
finally: 
    connection.close()


In [8]:
movie_tag_app = pd.DataFrame(list(result), columns=['movieId','tag','numApps','numPositive','numNeutral','numNegative','numDownvotes','score'])
movie_tag_app.head()

,movieId,tag,numApps,numPositive,numNeutral,numNegative,numDownvotes,score
0,1,2009 reissue in Stereoscopic 3-D,1,1,0,0,2,-1.0
1,1,3D,3,0,3,0,3,0.0
2,1,55 movies every kid should see--Entertainment ...,1,0,1,0,2,-1.0
3,1,action,1,0,1,0,1,0.0
4,1,action figure,1,0,1,0,2,-1.0


### Sum the scores of movie-tag which a user has applied to a movie. 
#### This is an naive approach that incorporates tag_quality into user_num_app


In [9]:
# movie_tag_app.groupby(["tag"]).sum().reset_index().sort_values(by="score",ascending=False).head(10)

In [10]:
user_movie_tag_score = pd.merge(user_tag_app2, movie_tag_app, on=['movieId','tag'], how='left')
#user_movie_tag_score.groupby(["tag"]).sum().reset_index()

In [11]:
# Sum the scores of tags which the user has applied.

user_movie_tag_score_agg = user_movie_tag_score.groupby(["tag"]).sum().reset_index().sort_values(by="score",ascending=False)

# Choose 10 as score threshold
# Exclude tags with a score < 10
user_movie_tag_score_agg = user_movie_tag_score_agg.loc[user_movie_tag_score_agg["score"]>=10]

# Note that this is the tag score agg only for one user
user_movie_tag_score_agg

,tag,movieId,user_num_app,affect,tstamp,numApps,numPositive,numNeutral,numNegative,numDownvotes,score
0,Hayao Miyazaki,11253,2,2,2,98,79,19,0,10,88.0
1,dystopia,3000,1,1,1,26,19,7,0,5,21.0
2,nature,3000,1,1,1,21,14,7,0,1,20.0


In [13]:
# Choose 10 as score threshold
# Exclude tags with a score < 10
user_movie_tag_score2 = user_movie_tag_score2.loc[user_movie_tag_score2["score"]>=10]

# Note that this is the tag score agg only for one user
user_movie_tag_score2.head(10)

NameError: name 'user_movie_tag_score2' is not defined

#### Filter out tags that are not in the candidate set

In [14]:
tag_genome_trimmed = pd.read_csv("Datasets/Tag_Genome_with_Score_Trimmed.csv")
tag_candidates = tag_genome_trimmed['tag'].tolist()

In [15]:
# Rank tags by scores, then by user_num_app
user_movie_tag_score_agg[user_movie_tag_score_agg['tag'].isin(tag_candidates)].sort_values(by=["score","user_num_app"],ascending=False).head(10)


,tag,movieId,user_num_app,affect,tstamp,numApps,numPositive,numNeutral,numNegative,numDownvotes,score
1,dystopia,3000,1,1,1,26,19,7,0,5,21.0
2,nature,3000,1,1,1,21,14,7,0,1,20.0


In [12]:
# Rank tags by user_num_app, then by scores
user_movie_tag_score_agg[user_movie_tag_score_agg['tag'].isin(tag_candidates)].sort_values(by=["user_num_app","score"],ascending=False).head(10)

,tag,movieId,user_num_app,affect,tstamp,numApps,numPositive,numNeutral,numNegative,numDownvotes,score
105,dystopia,109599,3,3,3,353.0,304.0,39.0,10.0,19.0,334.0
214,violence,61713,3,3,3,218.0,84.0,90.0,44.0,10.0,208.0
163,pixar,337419,3,3,3,124.0,75.0,44.0,5.0,3.0,121.0
215,violent,237654,3,3,3,91.0,36.0,23.0,32.0,6.0,85.0
71,cars,325142,3,3,3,30.0,16.0,14.0,0.0,7.0,23.0
79,classic,853,2,2,2,476.0,76.0,397.0,3.0,12.0,464.0
95,dark comedy,3154,2,2,2,382.0,333.0,41.0,8.0,25.0,357.0
84,computers,83034,2,2,2,173.0,151.0,22.0,0.0,8.0,165.0
208,time travel,13485,2,2,2,151.0,104.0,46.0,1.0,6.0,145.0
124,hackers,123208,2,2,2,86.0,71.0,11.0,4.0,5.0,81.0


### Aggregate table: userId, Tag, #Applied, Sum score, #Clicked/Searched

In [21]:
tag_search = pd.read_csv("tag_search_history_2.csv")
tag_search.rename(columns={"tagSearch":"tag"},inplace=True)

In [34]:
tag_search_43715 = tag_search[tag_search.userId == 43715]
tag_search_43715_agg = tag_search_43715.groupby(["userId","tag"]).count().reset_index()
tag_search_43715_agg.drop(["loginId","tstamp"],axis=1,inplace=True)
tag_search_43715_agg.rename(columns={"Unnamed: 0":"num_tag_search"},inplace=True)

In [40]:
user_43715_tag_activity_agg = pd.merge(user_movie_tag_score_agg, tag_search_43715_agg, on="tag", how="left")
user_43715_tag_activity_agg.drop(["Unnamed: 0"],axis=1,inplace=True)
user_43715_tag_activity_agg["num_tag_search"].fillna(0,inplace=True)
user_43715_tag_activity_agg

,tag,movieId,user_num_app,affect,numApps,numPositive,numNeutral,numNegative,numDownvotes,score,userId,num_tag_search
0,atmospheric,721621,70,70,2234.0,2099.0,126.0,9.0,184.0,2050.0,NaN,0.0
1,dark comedy,761642,35,35,1875.0,1729.0,123.0,23.0,150.0,1725.0,43715.0,2.0
2,twist ending,201032,11,11,1680.0,1533.0,121.0,26.0,117.0,1563.0,NaN,0.0
3,classic,187043,49,49,1687.0,822.0,836.0,29.0,214.0,1473.0,NaN,0.0
4,sci-fi,1042730,21,21,1519.0,1263.0,229.0,27.0,105.0,1414.0,43715.0,8.0
5,space,968060,19,19,1349.0,822.0,513.0,14.0,58.0,1291.0,NaN,0.0
6,surreal,518204,21,21,1342.0,1169.0,145.0,28.0,78.0,1264.0,NaN,0.0
7,psychology,223159,16,16,1197.0,1089.0,92.0,16.0,90.0,1107.0,43715.0,4.0
8,dystopia,697745,17,17,1118.0,925.0,168.0,25.0,84.0,1034.0,NaN,0.0
9,time travel,304322,15,15,952.0,690.0,228.0,34.0,39.0,913.0,NaN,0.0


Explanation of columns:
* user_num_app: how many times a given user (user43715 in this case) applied a tag
* numApps: how many times a given tag is aplied by all users
* numPositive: how many times users like this feature of the movie, across all movies
* numNeutral: how many times users feel neutral about this feature of the movie, across all movies
* numNegative: how many times users dislike this feature of the movie, across all movies
* numDownvotes: how many times users think it's a bad tag, across all movies
* score: sum of scores of tag of a movie, across movies a user has applied tag to
* userId;
* num_tag_search: number of time a given user searched for a tag

### Weight user-numAps by total numAps (Not a good approach)

In [47]:
# Consider: how to come up with global tag quality scores
movie_tag_app_sum = movie_tag_app.groupby("tag").sum().sort_values(by="numApps").reset_index()

In [48]:
movie_tag_app_sum.head(10)

,tag,movieId,numApps,numPositive,numNeutral,numNegative,numDownvotes,score
0,!950's Superman TV show,47950,1,0,1,0,5,-4.0
1,dna curse,1653,1,0,1,0,2,-1.0
2,dna testing,139245,1,0,1,0,0,1.0
3,do NOT like hugh grant,357,1,0,1,0,0,1.0
4,do kupienia,52975,1,0,1,0,0,1.0
5,do not watch,133749,1,0,0,1,0,1.0
6,do zassania HD,132874,1,0,1,0,0,1.0
7,doc,42179,1,0,1,0,0,1.0
8,doc actor,147144,1,0,1,0,0,1.0
9,docker,65660,1,0,1,0,0,1.0


In [66]:
movie_tag_app_dict = {}
tag_score_dict = {}

for index, row in movie_tag_app_sum.iterrows():
    tag = row[0]
    numApps = row[2]
    score = row[7]
    movie_tag_app_dict[tag] = numApps
    tag_score_dict[tag] = score

In [80]:
user_tag_app_agg = user_tag_app2.groupby("tag").sum().reset_index()
user_tag_app_agg.drop(columns=["movieId","affect","score"],inplace=True)

In [81]:
# This block need debug

# These three lists will be added as a column to user_tag_app
total_num_apps = []
weighted_num_apps = []

for index, row in user_tag_app_agg.iterrows():
    tag = row[0]
    if  tag in movie_tag_app_dict:
        total_num_apps.append(movie_tag_app_dict[tag])
    else:
        total_num_apps.append(row[1])

In [82]:
user_tag_app_agg["total_num_apps"] = total_num_apps
user_tag_app_agg["weighted_num_apps"] = user_tag_app_agg["user_num_app"]/user_tag_app_agg["total_num_apps"]

In [86]:
user_tag_app_agg.sort_values(by="user_num_app",ascending=False).head(10)

,tag,user_num_app,total_num_apps,weighted_num_apps
1664,adapted from:book,140,1209,0.115798
1232,R,120,816,0.147059
188,Bechdel Test:Fail,86,529,0.162571
1052,PG-13,80,195,0.410256
190,Bechdel Test:Pass,72,511,0.140900
1802,atmospheric,70,5984,0.011698
938,New York City,66,888,0.074324
3325,movielens top pick,57,72,0.791667
444,Edgar Award (Best Motion Picture),54,79,0.683544
1264,R:language,54,66,0.818182
